# Data Engineer technical assessment (stock)
# Goal

The *Indiretail* company has asked us to create an application that calculates its daily stock and a series of KPIs, in order to know how has been the evolution of stock and sales in their company and the impact of COVID.

The following data is provided for 2019 and 2020:

- Stock movements: inbounds are positive values and outbounds are negative values.
- Sales
- Product master

# Data

The data provided by the company are as follows:

## Stock movements

*stock_movements.parquet* file with the following information:

- StoreId (integer): store code.
- ProductId (integer): product code.
- Date (date): inbound/outbound date.
- Quantity (integer): inbound (+) or outbound (-).

## Sales

*sales.parquet* file with the following information:

- StoreId (integer): store code.
- ProductId (integer): product code.
- Date (date): date of sale.
- Quantity (integer): units sold (negative values are returns).

## Product master

*products.parquet* file with the following information:

- ProductRootCode (integer): product root code, without size.
- ProductId (integer): product code.
- SupplierPrice (float): product cost.
- RetailPrice (float): retail price of the product.
- Family (string): product family to which it belongs.

# What do we ask?

*Indiretail* ask for the following calculations.

## Daily stock calculation

From the stock movements you have to calculate the daily stock since 01/01/2019, at product root code level. For the calculation:

- Join the product root code table to the stock movements table.

  **IMPORTANT**: products with stock movements that do not have a root code assigned must be deleted.

- Group the table by product root code, store and date, adding the units. Example:  

   ![image](https://user-images.githubusercontent.com/7293776/145417614-6a682746-51cf-4f5b-a788-7424bae37e88.png)

- Accumulate the stock movements to obtain the stock on that date, by product root code, store and date level. Example:

   ![image](https://user-images.githubusercontent.com/7293776/145417461-1445d2c4-25f4-4a56-9314-a0d16af353ba.png)

- Calculate the initial stock of each product root code - store on 01/01/2019 and maintain the dates after 01/01/2019 and its stock. Example:

   ![image](https://user-images.githubusercontent.com/7293776/145417374-acb7fa79-08e2-48a8-ad4a-38df3a579e7f.png)

In the following cases the key, store - product root codes, can be removed from the table:

- With only initial stock and value 0, since they have no stock from 2019, the case of the example in store 2.
- With negative stock on all dates, the case of the example in store 1.

The customer asks us for the above information in two formats:

- For each product root code - store, the date ranges and the stock in that range. the date, 31/12/2020, is considered as the last date. Example:

  ![image](https://user-images.githubusercontent.com/7293776/145400537-2540a71f-4cc4-4fa8-b15d-6747b0935cac.png)

In store 12 and 28 there is only one date and with stock 3, this means that this product has not been sold, nor has it more inbounds, thus maintaining a stock of 3 from 01/01/2019 until the last day of 2020 (31/12/2020).

In the case of store 14 there are two dates:

    - 01/01/2019: stock of 1

    - 27/10/2019: stock of 0

The product had stock of 1 in the store until 26/10/2019, as the sale of the only unit in stock occurs on 27/10/2019. Therefore, only one interval appears, from 01/01/2019 to 26/10/2019.

The intervals with negative stock or 0 are removed from the table.

A .parquet file named ***interval_stock*** will be delivered.

- The daily stock propagated to all dates. Example:

  ![image](https://user-images.githubusercontent.com/7293776/145414176-17ea2c03-149c-4584-a533-7c26df316c7a.png)
  
For stores 12 and 28, we have to generate all dates from 01/01/2019 to 31/12/2020 with stock at 3. 

For store 14 only from 01/01/2019 to 26/10/2019, same as before, we do not take into account dates with negative or 0 stock.

A .parquet file with name ***daily_stock*** will be delivered.

## KPIs

The customer asks us to calculate the following KPIs:

- Profit. The calculation is as follows:

  ![image](https://user-images.githubusercontent.com/7293776/138304517-932c8838-ed6a-4fe6-93ce-528504c885b7.png)

We use the files *sales.parquet*, *products.parquet* and *stock_movements.parquet*.

The retail price of the product is the column *RetailPrice* and the cost of the product is the field *SupplierPrice,* both are in the file *products.parquet*.

The units sold are obtained from the file *sales.parquet*.

The inbounds are obtained from the file *stock_movements.parquet*, we will take into account the positive values of the field *Quantity*.

Both in the file *sales.parquet* and *stock_movements.parquet* it is not necessary to take into account products that are not in the master *products.parquet*.

- Turnover. The calculation is as follows:

  ![image](https://user-images.githubusercontent.com/7293776/138304810-bcf698d7-e413-4c2f-89af-991a16abefba.png)

We use the files *sales.parquet* and *daily_stock* (one of the files you have created).

The units sold are obtained from the file *sales.parquet*.

The stock is obtain from the file *daily_stock*.

Both in the file *sales.parquet* and *daily_stock* it is not necessary to take into account products that are not in the master *products.parquet*.

Both KPIs must be delivered:

- At store and year level, for 2019 and 2020.
- At product family and year level, for 2019 and 2020.

Several .parquet files will be delivered with name ***store_rotation***, ***family_rotation***, ***store_benefits***, ***family_benefits***.

# Presentation

Develop an application in Python, which reads the .parquet files and generates the files requested by *Indiretail*:

- Daily Stock: ***interval_stock*** and ***daily_stock***

- KPIs: ***store_rotation***, ***family_rotation***, ***store_benefits***, ***family_benefits***.

*Due to the large volume of Indiretail data, the code will run in a cluster on Azure Databricks, so the code must be able to run on Spark.*

The code has to be uploaded to a GitHub repository with the name PruebaIndiretail_order_*{YourName}*.

In addition to well-written code, following best practices, we care about processing time and resource consumption (RAM, CPU, etc.) keep this in mind!

If you use the language wisely, it will help you with configuration, dependency injection, logging, etc.

You will be able to use the libraries you need.

We would also like to know the reasons for the decisions you made (and also the ones you discarded). If you go with a good README.md, we would be very happy!

Finally, if you consider necessary to add some automated testing to gain more confidence, it never hurts!